In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
import pickle
!pip install streamlit


ModuleNotFoundError: No module named 'pandas'

In [ ]:
car_data = pd.read_csv('quikr_car.csv')
car_data.head()


,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,"80,000","45,000 kms",Petrol
1,Mahindra Jeep CL550 MDI,Mahindra,2006,"4,25,000",40 kms,Diesel
2,Maruti Suzuki Alto 800 Vxi,Maruti,2018,Ask For Price,"22,000 kms",Petrol
3,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,"3,25,000","28,000 kms",Petrol
4,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,"5,75,000","36,000 kms",Diesel


In [ ]:
car_data.shape

(892, 6)

In [ ]:
car_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        892 non-null    object
 1   company     892 non-null    object
 2   year        892 non-null    object
 3   Price       892 non-null    object
 4   kms_driven  840 non-null    object
 5   fuel_type   837 non-null    object
dtypes: object(6)
memory usage: 41.9+ KB


In [ ]:
car_data.isnull().sum()

,0
name,0
company,0
year,0
Price,0
kms_driven,52
fuel_type,55


In [ ]:
car_data.duplicated().sum()

94

**DATA-PRE PROCESSING**

-->Changes to be made in data


*  year has many non-year values
*  year object to int
*  price has ask for price
*  price object to int
*  kms_driven  has kms with integers
*  kms_driven object to int
*  kms_driven has null values
*  fuel_type has null values
*  only first 3 words of name

In [ ]:
# making a copy of the data
backup = car_data.copy()

In [ ]:
#removing the ask for price from price column
car_data = car_data[car_data['Price']!='Ask For Price']

In [ ]:
#removing the comma and conerting the value type from objrct to int
car_data['Price'] = car_data['Price'].astype(str).str.replace(',','')

#also changing the value type from object to int
car_data['Price'] = car_data['Price'].astype(int)

<ipython-input-52-3a43f8e43518>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  car_data['Price'] = car_data['Price'].astype(str).str.replace(',','')
<ipython-input-52-3a43f8e43518>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  car_data['Price'] = car_data['Price'].astype(int)


In [ ]:
#kms_driven has kms included
#converting the value type from object to int
car_data['kms_driven'] = car_data['kms_driven'].astype(str).str.replace(' kms','')

car_data['kms_driven'] = car_data['kms_driven'].astype(str).str.replace(',','')

car_data = car_data[car_data['kms_driven'].str.isnumeric()]

#also changing the value type from object to int
car_data['kms_driven'] = car_data['kms_driven'].astype(int)



<ipython-input-53-38e237bdb374>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  car_data['kms_driven'] = car_data['kms_driven'].astype(str).str.replace(' kms','')
<ipython-input-53-38e237bdb374>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  car_data['kms_driven'] = car_data['kms_driven'].astype(str).str.replace(',','')


In [ ]:
car_data = car_data[~car_data['fuel_type'].isna()]

In [ ]:
#splitting with respect to space and slicing first 3 words and again join them
car_data['name'] = car_data['name'].str.split(' ').str.slice(0,3).str.join(' ')

In [ ]:
car_data # if you observe carefully, our indexes are not correct

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing,Hyundai,2007,80000,45000,Petrol
1,Mahindra Jeep CL550,Mahindra,2006,425000,40,Diesel
3,Hyundai Grand i10,Hyundai,2014,325000,28000,Petrol
4,Ford EcoSport Titanium,Ford,2014,575000,36000,Diesel
6,Ford Figo,Ford,2012,175000,41000,Diesel
...,...,...,...,...,...,...
883,Maruti Suzuki Ritz,Maruti,2011,270000,50000,Petrol
885,Tata Indica V2,Tata,2009,110000,30000,Diesel
886,Toyota Corolla Altis,Toyota,2009,300000,132000,Petrol
888,Tata Zest XM,Tata,2018,260000,27000,Diesel


In [ ]:
car_data = car_data.reset_index(drop=True) #here drop() drops the wrong index
car_data

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing,Hyundai,2007,80000,45000,Petrol
1,Mahindra Jeep CL550,Mahindra,2006,425000,40,Diesel
2,Hyundai Grand i10,Hyundai,2014,325000,28000,Petrol
3,Ford EcoSport Titanium,Ford,2014,575000,36000,Diesel
4,Ford Figo,Ford,2012,175000,41000,Diesel
...,...,...,...,...,...,...
811,Maruti Suzuki Ritz,Maruti,2011,270000,50000,Petrol
812,Tata Indica V2,Tata,2009,110000,30000,Diesel
813,Toyota Corolla Altis,Toyota,2009,300000,132000,Petrol
814,Tata Zest XM,Tata,2018,260000,27000,Diesel


In [ ]:
car_data[car_data['Price']>6e6] # there is only one car above 6 lakhs so we will remove it

,name,company,year,Price,kms_driven,fuel_type
534,Mahindra XUV500 W6,Mahindra,2014,8500003,45000,Diesel


In [ ]:
car_data = car_data[car_data['Price']<6e6].reset_index(drop=True)
car_data

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing,Hyundai,2007,80000,45000,Petrol
1,Mahindra Jeep CL550,Mahindra,2006,425000,40,Diesel
2,Hyundai Grand i10,Hyundai,2014,325000,28000,Petrol
3,Ford EcoSport Titanium,Ford,2014,575000,36000,Diesel
4,Ford Figo,Ford,2012,175000,41000,Diesel
...,...,...,...,...,...,...
810,Maruti Suzuki Ritz,Maruti,2011,270000,50000,Petrol
811,Tata Indica V2,Tata,2009,110000,30000,Diesel
812,Toyota Corolla Altis,Toyota,2009,300000,132000,Petrol
813,Tata Zest XM,Tata,2018,260000,27000,Diesel


In [ ]:
car_data.to_csv('Cleaned_Car_data.csv')

In [ ]:
#working on model
#separating in two
X = car_data.drop(columns='Price')
Y = car_data['Price']

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [ ]:
#One-hot encoding (OHE) is a machine learning technique that converts categorical data into numerical values so that machines can understand it. It's useful for algorithms that can't work with categorical data directly, like logistic regression.
ohe = OneHotEncoder()
ohe.fit(X[['name', 'company', 'fuel_type']])

OneHotEncoder()

In [ ]:
column_transform = make_column_transformer((OneHotEncoder(categories=ohe.categories_), ['name', 'company', 'fuel_type']),
                                           remainder='passthrough') # remainder = other column

In [ ]:
model = LinearRegression()

In [ ]:
pipe = make_pipeline(column_transform, model)

In [ ]:
pipe.fit(X_train, Y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Audi A3 Cabriolet', 'Audi A4 1.8', 'Audi A4 2.0', 'Audi A6 2.0',
       'Audi A8', 'Audi Q3 2.0', 'Audi Q5 2.0', 'Audi Q7', 'BMW 3 Series',
       'BMW 5 Series', 'BMW 7 Series', 'BMW X1', 'BMW X1 sDrive20d',
       'BMW X1 xDrive20d', 'Chevrolet Beat', 'Chevrolet Beat...
                                                                            array(['Audi', 'BMW', 'Chevrolet', 'Datsun', 'Fiat', 'Force', 'Ford',
       'Hindustan', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Land',
       'Mahindra', 'Maruti', 'Mercedes', 'Mini', 'Mitsubishi', 'Nissan',
       'Renault', 'Skoda', 'Tata', 'Toyota', 'Volkswagen', 'Volvo'],
      dtype=object),
                                                                            array(['Diesel', 'LPG', 'Petrol'], dtype=object)]),
                                                  ['name', 'company',
                                                   'fuel_type'])])),
                ('linearregression', LinearRegression())])

In [ ]:
prediction = pipe.predict(X_test)
r2_score = r2_score(Y_test, prediction)
print(r2_score)

0.6206184325887315


In [ ]:
#getting the highest r2_score
from sklearn.metrics import r2_score # Import the r2_score function

score = []
for i in range(1000):
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=i)
  model = LinearRegression()
  pipe = make_pipeline(column_transform, model)
  pipe.fit(X_train, Y_train)
  prediction = pipe.predict(X_test)
  score.append(r2_score(Y_test, prediction))

In [ ]:
score[np.argmax(score)]

0.8457059012561223

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=np.argmax(score))
model = LinearRegression()
pipe = make_pipeline(column_transform, model)
pipe.fit(X_train, Y_train)
prediction = pipe.predict(X_test)
r2_score(Y_test, prediction)

0.8457059012561223

In [ ]:
pickle.dump(pipe, open('LinearRegressionModel.pkl', 'wb'))

In [ ]:
pipe.predict(pd.DataFrame([['Maruti Suzuki Swift', 'Maruti', 2019, 100, 'Petrol']], columns=['name', 'company', 'year', 'kms_driven', 'fuel_type']))

array([458894.10960853])

In [ ]:
company = car_data['company'].unique()
pickle.dump(company, open('company.pkl', 'wb'))

In [ ]:
car_model = car_data['name'].unique()
pickle.dump(car_model, open('car_model.pkl', 'wb'))

In [ ]:
year = car_data['year'].unique()
year = sorted(year, reverse=True)
pickle.dump(year, open('year.pkl', 'wb'))

In [ ]:
kms_driven = car_data['kms_driven']
pickle.dump(kms_driven, open('kms_driven.pkl', 'wb'))
kms_driven

,kms_driven
0,45000
1,40
2,28000
3,36000
4,41000
...,...
810,50000
811,30000
812,132000
813,27000


In [ ]:
fuel_type = car_data['fuel_type']
pickle.dump(fuel_type, open('fuel_type.pkl', 'wb'))
fuel_type

,fuel_type
0,Petrol
1,Diesel
2,Petrol
3,Diesel
4,Diesel
...,...
810,Petrol
811,Diesel
812,Petrol
813,Diesel


In [ ]:
#saving the trained model
import pickle
filename = 'trained_model.sav'
pickle.dump(pipe, open(filename, 'wb'))

In [ ]:
%%writefile app.py
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
import numpy as np
import streamlit as st
import pandas as pd
import pickle
import math


#title
st.title('Car Price Predictor')

#loading the data
company = pickle.load(open('company.pkl', 'rb')) #rb=ReadBinary
car_model = pickle.load(open('car_model.pkl', 'rb')) #rb=ReadBinary
kms_driven = pickle.load(open('kms_driven.pkl', 'rb')) #rb=ReadBinary
fuel_type = pickle.load(open('fuel_type.pkl', 'rb')) #rb=ReadBinary
year = pickle.load(open('year.pkl', 'rb')) #rb=ReadBinary

#loading the model
loaded_model = pickle.load(open('trained_model.sav', 'rb'))




company_selected = st.selectbox(
    'Choose the Company',
    (company)
)
car_model_selected = st.selectbox(
    'Choose the model',
    (car_model)
)
kms_driven_selected = st.text_input(
    'Kilometers driven',
    ''
)

fuel_type_selected = st.selectbox(
    'Select the Fuel_type',
    ('Petrol', 'Diesel', 'LPG')
)
year_selected = st.selectbox(
    'Select the year',
    (year)
)


#creating a function for predicting
def car_price_prediction():
    final_prediction = loaded_model.predict(pd.DataFrame([[car_model_selected, company_selected, year_selected, kms_driven_selected, fuel_type_selected]],
                                                         columns=['name', 'company', 'year', 'kms_driven', 'fuel_type']))
    final_prediction = math.trunc(int(final_prediction))
    st.write(f'The Price Of The Car is:- {final_prediction}')


if st.button('Show the Price'):
    car_price_prediction()
else:
    pass


Overwriting app.py


In [ ]:
!wget -q -O - ipv4.icanhazip.com # copy the code below and paste it in the 'your url is:' generated in the below cell

34.86.4.217


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.86.4.217:8501

your url is: https://light-clouds-beam.loca.lt
/content/app.py:57: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  final_prediction = math.trunc(int(final_prediction))
/content/app.py:57: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  final_prediction = math.trunc(int(final_prediction))
  Stopping...
^C
